## ZodFrames & ZodSequences: an introduction
This notebook aims to introduce the ZodFrames & ZodSequences classes, which are helper classes to interact with the Frames and Sequences subsets of Zenseact Open Dataset (ZOD). It will highlight some basic functionality that can be used to build dataloaders in various frameworks, for example PyTorch.

In [ ]:
# Import the package
from zod import ZodFrames, ZodSequences
from zod.data_classes import LidarData
import zod.constants as constants
from zod.constants import Lidar, Anonymization

In [ ]:
# Initialize the class with the dataset root and version, version can be "mini" or "full"
dataset_root = ""  # NOTE! Set this to the path to the dataset
zod_frames = ZodFrames(dataset_root=dataset_root, version="mini")
zod_sequences = ZodSequences(dataset_root=dataset_root, version="mini")

In [ ]:
# Get the training and validation split
training_frames = zod_frames.get_split(constants.TRAIN)
validation_frames = zod_frames.get_split(constants.VAL)

# Print the number of training and validation frames
print(f"Number of training frames: {len(training_frames)}")
print(f"Number of validation frames: {len(validation_frames)}")

# Print the number of training and validation sequences
training_sequences = zod_sequences.get_split(constants.TRAIN)
validation_sequences = zod_sequences.get_split(constants.VAL)
print(f"Number of training sequences: {len(training_sequences)}")
print(f"Number of validation sequences: {len(validation_sequences)}")

# Print out the first 5 training frames
print("The 5 first training frames have the ids:", training_frames[:5])

# Show the first training sequence
print("The first training sequence has the id:", training_sequences[0])

In [ ]:
# We can also get a frame by its id
frame_from_id = zod_frames["009158"]

# We can also get it via the index
frame_from_idx = zod_frames[9158]

# The two frames should be the same
assert frame_from_id.info == frame_from_idx.info

In [ ]:
# We can use the frame to get the ego-motion of the vehicle
ego_motion = frame_from_id.ego_motion
print(f"Acceleration: {ego_motion.accelerations.shape}")
print(f"Velocities: {ego_motion.velocities.shape}")
print(f"Poses: {ego_motion.poses.shape}")
print(f"Timestamps: {ego_motion.timestamps.shape}")

In [ ]:
# Note that the ego-motion is a lightweight version of the oxts data
oxts = frame_from_id.oxts
print(f"Acceleration: {oxts.accelerations.shape}")
print(f"Velocities: {oxts.velocities.shape}")
print(f"Poses: {oxts.poses.shape}")
print(f"Timestamps: {oxts.timestamps.shape}")

Note that the ZodFrames class yield a `ZodFrame` which acts a cache for the lightweight data (e.g., ego-motion, calibration, and metadata), but also holds an `info` attribute. This in turn holds all the paths to more heavyweight data (e.g., images and point clouds).

In [ ]:
# Get the lidar core-frame for this ZodFrame
lidar_core_frame = frame_from_id.info.get_key_lidar_frame()
print(lidar_core_frame)

In [ ]:
# Load the lidar data
pc: LidarData = lidar_core_frame.read()
print(f"Points: {pc.points.shape}")  # x, y, z
print(f"Timestamps: {pc.timestamps.shape}")
print(f"Intensity: {pc.intensity.shape}")
print(f"Diode: {pc.diode_idx.shape}")

In [ ]:
# We can also iterate over all point-clouds
lidar_frames = frame_from_id.info.get_lidar_frames()
for i, lidar_frame in enumerate(lidar_frames):
    pc = lidar_frame.read()
    print(f"Lidar frame # {i}")
    print(f"Points: {pc.points.shape}")  # x, y, z
    print("-" * 20)

In [ ]:
# We can do the same for the sequences
seq = zod_sequences[training_sequences[0]]

# Get the lidar frames
print(f"Number of lidar frames: {len(seq.info.get_lidar_frames(lidar=Lidar.VELODYNE))}")
# We can also get the original camera frames
print(
    f"Number of camera frames: {len(seq.info.get_camera_frames(anonymization=Anonymization.ORIGINAL))}"
)

# Or see how long the sequence is
print(f"Timespan: {(seq.info.end_time - seq.info.start_time).total_seconds()}")